In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
import json
import googlemaps
import folium

# Import API key
from api_keys import geoapify_key
from api_keys import API_KEY


<div style="text-align: center;">
    
# FIRST PART
---

</div>

## LOAD TORTILLA PRICES CSV, GEOAPIFY CONNECTION AND CREATION OF DATAFRAME
### IN THIS PART WE WILL LOAD THE CSV AND THEN GENERATE A DATAFRAME WITH GEAPIFY API IN ORDER TO VISUALIZE THE CITIES WITH HIGHEST AND LOWEST PRICES OF TORTILLA IN MEXICO IN 2024


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
tortilla_df = pd.read_csv("Resources/new_tortilla.csv")
tortilla_df


,State,State.1,City,Year,Month,Day,Store type,Price per kilogram
0,113484,Aguascalientes,Aguascalientes,2014,1,3,Mom and Pop Store,12.00
1,113485,Baja California,Mexicali,2014,1,3,Mom and Pop Store,16.29
2,113486,Baja California,Tijuana,2014,1,3,Mom and Pop Store,13.55
3,113487,Baja California Sur,La Paz,2014,1,3,Mom and Pop Store,14.25
4,113488,Campeche,Campeche,2014,1,3,Mom and Pop Store,14.50
...,...,...,...,...,...,...,...,...
170594,289141,Veracruz,Coatzacoalcos,2024,10,21,Big Retail Store,12.57
170595,289142,Veracruz,Veracruz,2024,10,21,Big Retail Store,13.27
170596,289143,Veracruz,Xalapa,2024,10,21,Big Retail Store,13.73
170597,289144,Yucatán,Mérida,2024,10,21,Big Retail Store,13.13


In [3]:
#selectind the data just from 2024 since is the data of interest for the map
tortilla_2024 = tortilla_df[tortilla_df['Year'] >= 2024]
tortilla_2024

,State,State.1,City,Year,Month,Day,Store type,Price per kilogram
157641,276186,Aguascalientes,Aguascalientes,2024,1,3,Mom and Pop Store,21.67
157642,276187,Baja California,Mexicali,2024,1,3,Mom and Pop Store,30.57
157643,276188,Baja California,Tijuana,2024,1,3,Mom and Pop Store,25.64
157644,276189,Baja California Sur,La Paz,2024,1,3,Mom and Pop Store,27.00
157645,276190,Campeche,Campeche,2024,1,3,Mom and Pop Store,26.00
...,...,...,...,...,...,...,...,...
170594,289141,Veracruz,Coatzacoalcos,2024,10,21,Big Retail Store,12.57
170595,289142,Veracruz,Veracruz,2024,10,21,Big Retail Store,13.27
170596,289143,Veracruz,Xalapa,2024,10,21,Big Retail Store,13.73
170597,289144,Yucatán,Mérida,2024,10,21,Big Retail Store,13.13


In [4]:
#calculating how many cities are in the data
distinct_cities_count = tortilla_2024['City'].nunique()
distinct_cities_count

54

In [5]:
#getting an average price per city
average_price_city = tortilla_2024.groupby(['State.1', 'City'])['Price per kilogram'].mean().reset_index()
average_price_city.rename(columns={'Price per kilogram': 'Average Price per Kilogram'}, inplace=True)
average_price_city.head(10)

,State.1,City,Average Price per Kilogram
0,Aguascalientes,Aguascalientes,17.558333
1,Baja California,Mexicali,22.890458
2,Baja California,Tijuana,20.395083
3,Baja California Sur,La Paz,20.153917
4,Campeche,Campeche,19.118083
5,Chiapas,Tapachula,18.749125
6,Chiapas,Tuxtla Gutiérrez,17.483333
7,Chihuahua,Cd. Juárez,20.221417
8,Chihuahua,Chihuahua,22.650250
9,Coahuila,Piedras Negras,24.100417


In [6]:
#creating a new dataframe to get the lat and Long from the Geapify API
average_price_city["Lat"] = ""
average_price_city["Lon"] = ""
average_price_city.head(10)

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Aguascalientes,Aguascalientes,17.558333,,
1,Baja California,Mexicali,22.890458,,
2,Baja California,Tijuana,20.395083,,
3,Baja California Sur,La Paz,20.153917,,
4,Campeche,Campeche,19.118083,,
5,Chiapas,Tapachula,18.749125,,
6,Chiapas,Tuxtla Gutiérrez,17.483333,,
7,Chihuahua,Cd. Juárez,20.221417,,
8,Chihuahua,Chihuahua,22.650250,,
9,Coahuila,Piedras Negras,24.100417,,


In [7]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [8]:
# Print a message to follow up the city search
print("Starting city search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in average_price_city.iterrows():

    # Get the city's name & add ",Mexico" to the string so geoapify finds the correct city
    city = row["City"] + ", México"

    # Add the current city to the parameters
    params["text"] = f"{city}"

    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Convert response to JSON
    response = response.json()

    # Extract latitude and longitude
    average_price_city.loc[index, "Lat"] = response["results"][0]["lat"]
    average_price_city.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
average_price_city.head()

Starting city search
Coordinates for Aguascalientes, México fetched...
Coordinates for Mexicali, México fetched...
Coordinates for Tijuana, México fetched...
Coordinates for La Paz, México fetched...
Coordinates for Campeche, México fetched...
Coordinates for Tapachula, México fetched...
Coordinates for Tuxtla Gutiérrez, México fetched...
Coordinates for Cd. Juárez, México fetched...
Coordinates for Chihuahua, México fetched...
Coordinates for Piedras Negras, México fetched...
Coordinates for Saltillo, México fetched...
Coordinates for Torreón, México fetched...
Coordinates for Colima, México fetched...
Coordinates for D.F., México fetched...
Coordinates for ZM D.F., México fetched...
Coordinates for Durango, México fetched...
Coordinates for Gómez Palacio, México fetched...
Coordinates for Toluca, México fetched...
Coordinates for Celaya, México fetched...
Coordinates for León, México fetched...
Coordinates for Acapulco, México fetched...
Coordinates for Chilpancingo, México fetched..

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Aguascalientes,Aguascalientes,17.558333,21.880487,-102.296719
1,Baja California,Mexicali,22.890458,32.640525,-115.474899
2,Baja California,Tijuana,20.395083,32.53174,-117.019529
3,Baja California Sur,La Paz,20.153917,24.161995,-110.315853
4,Campeche,Campeche,19.118083,19,-90.5


<div style="text-align: center;">
    
# SECOND PART
---

</div>

## MAPS GENERATION WITH HVPLOTS
### IN THIS PART WE WILL GENERATE SOME MAPS WITH THE HVPLOTS LIBRARY

In [9]:
#creating a map with HV PLOTS to display the cities found sizing by price
# Configure the map plot
map_plot_1 = average_price_city.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Average Price per Kilogram",
    scale = 2.5,
    color = "City",
    hover_cols = ["State.1", "City"],
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Average Price per Kilogram,State.1)

In [10]:
#sorting the price from highest to lowest
sorted_price = average_price_city.sort_values(by='Average Price per Kilogram', ascending=False)
sorted_price_index = sorted_price.reset_index(drop=True)
sorted_price_index.head(15)

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Coahuila,Piedras Negras,24.100417,28.704596,-100.516714
1,Tamaulipas,Matamoros,23.221708,25.463614,-105.432334
2,Baja California,Mexicali,22.890458,32.640525,-115.474899
3,Sonora,Hermosillo,22.725750,29.094821,-110.96922
4,Chihuahua,Chihuahua,22.650250,25.543477,-99.955224
5,Sonora,San Luis Río Colorado,21.984708,32.451796,-114.765254
6,Tamaulipas,Reynosa,21.983250,26.090767,-98.278819
7,Guerrero,Acapulco,21.671292,17.962075,-94.761641
8,Sonora,Cd. Obregón,21.464292,27.484654,-109.935961
9,Tamaulipas,Nuevo Laredo,21.424750,15.563038,-92.385844


In [11]:
#get the seven most expensive cities
top_7_cities = sorted_price_index.head(7)
top_7_cities

,State.1,City,Average Price per Kilogram,Lat,Lon
0,Coahuila,Piedras Negras,24.100417,28.704596,-100.516714
1,Tamaulipas,Matamoros,23.221708,25.463614,-105.432334
2,Baja California,Mexicali,22.890458,32.640525,-115.474899
3,Sonora,Hermosillo,22.725750,29.094821,-110.96922
4,Chihuahua,Chihuahua,22.650250,25.543477,-99.955224
5,Sonora,San Luis Río Colorado,21.984708,32.451796,-114.765254
6,Tamaulipas,Reynosa,21.983250,26.090767,-98.278819


In [12]:
# Plotting the Seven most expensive cities
map_plot_2 = top_7_cities.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Average Price per Kilogram",
    scale = 2.5,
    color = "City",
    hover_cols = ["State.1", "City"],
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Average Price per Kilogram,State.1)

In [13]:
# Get the seven cheapest cities
bottom_7_cities = sorted_price_index.tail(7)

bottom_7_cities

,State.1,City,Average Price per Kilogram,Lat,Lon
47,Chiapas,Tuxtla Gutiérrez,17.483333,16.753801,-93.115959
48,D.F.,D.F.,17.362500,19.43263,-99.133178
49,D.F.,ZM D.F.,17.011625,16.66806,-92.56861
50,Edo. México,Toluca,16.930000,19.292545,-99.656901
51,Puebla,Puebla,16.186125,16.91442,-92.503107
52,Tlaxcala,Tlaxcala,15.352417,19.416667,-98.166667
53,Puebla,ZM Puebla,15.145833,20.005045,-97.693422


In [14]:
# Plotting the seven cheapest cities
map_plot_3 = bottom_7_cities.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    size = "Average Price per Kilogram",
    scale = 2.5,
    color = "City",
    hover_cols = ["State.1", "City"],
)

# Display the map plot
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Average Price per Kilogram,State.1)

<div style="text-align: center;">
    
# THIRD PART
---

</div>

## GOOGLE MAPS PLACES API CONNECTION AND MAPS GENERATION WITH FOLIUM
### IN THIS PART WE WILL CONNECT TO GOOGLE MAPS PLACES API IN ORDER TO GET THE LAN AND LONG OF DIFFERENT "TORTILLERIAS" OVER CITIES OF INTEREST THROUGH MEXICO. THEN WE WILL USE THE FOLIUM LIBRARY TO HAVE INTERACTIVE MAPS WITH THE RESULTS

In [15]:
#-------------------------------------------------------------------------------------------------------------------
# Setting the google maps places API and checking connection
#-------------------------------------------------------------------------------------------------------------------

# Define the base URL for the Google Places API
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# Create a search query for tortillerías in Mexico
query = 'tortillerías in Mexico'

# Set up the parameters for the API request
params = {
    'query': query,
    'key': API_KEY
}

# Make the request to the Google Places API
response = requests.get(base_url, params=params)
response

<Response [200]>

In [26]:
#-------------------------------------------------------------------------------------------------------------------
# In this part we will connect to Google Maps API in order to get data from "Tortillerias" near the cities
# that we are interested in. In this case those will be the 7 cities with highest tortilla price in 2024
#-------------------------------------------------------------------------------------------------------------------
gmaps = googlemaps.Client(key=API_KEY)

locations = []

# Function to check if a location is within Mexico's bounds
def is_within_mexico(lat, lng):
    mexico_bounds = {
        'lat_min': 14.5320,  # Minimum latitude
        'lat_max': 32.7187,  # Maximum latitude
        'lng_min': -118.4043, # Minimum longitude
        'lng_max': -86.7106   # Maximum longitude
    }
    
    return (mexico_bounds['lat_min'] <= lat <= mexico_bounds['lat_max']) and (mexico_bounds['lng_min'] <= lng <= mexico_bounds['lng_max'])

# Looping through the rows of top_7_cities to fetch tortillerías in a radious of 20000 m
for index, row in top_7_cities.iterrows():
    estado = row['State.1']
    coords = (row['Lat'], row['Lon'])
    query = f'tortillería {estado}'
    
    # Query the Google Places API for tortillerías in the area
    results = gmaps.places(query=query, location=coords, radius=20000, region="MX")
    
    for place in results.get('results', []):
        lat = place['geometry']['location']['lat']
        lng = place['geometry']['location']['lng']
        
        # Check if the location is within the bounds of Mexico
        if is_within_mexico(lat, lng):  # Filter by Mexico's limits
            locations.append({
                'name': place['name'],
                'lat': lat,
                'lng': lng,
                'estado': estado
            })
        else:
            print(f"Filtered out place: {place['name']} at ({lat}, {lng}) - Outside of Mexico")

# Create a DataFrame for tortillerías
tortillerias_df = pd.DataFrame(locations)

# Display the resulting DataFrame
tortillerias_df.head(50)

Filtered out place: Tortilleria Tamaulipas at (31.1546833, -83.7353766) - Outside of Mexico


,name,lat,lng,estado
0,Tortilleria COAHUILA,20.701395,-103.352564,Coahuila
1,Tortillería Alejandra,25.521601,-103.430551,Coahuila
2,Maizito Tortillería,25.557959,-103.387671,Coahuila
3,Tortillería La Paloma,25.427446,-100.996524,Coahuila
4,Tortillería La Nueva,28.718160,-100.520536,Coahuila
5,Tortillería Cinthia Sucursal Juárez,25.538718,-103.348329,Coahuila
6,Tortillería El Rayito,25.531533,-103.432695,Coahuila
7,Tortilleria Coahuila,32.626761,-115.440086,Coahuila
8,Tortillería Acapulco,25.525377,-103.360143,Coahuila
9,Tortillería Torreón,25.543070,-103.452336,Coahuila


In [34]:
exclude_names = ['Tortilleria Tamaulipas', 'Tortillería Tamaulipas', 'Tortilleria COAHUILA', 'Tortilleria tamaulipas #2', 'Tortilleria Tamaulipas #3']
tortillerias_df_dropped = tortillerias_df[~tortillerias_df['name'].isin(exclude_names)]
tortillerias_df_dropped.tail(50)


,name,lat,lng,estado
27,"Tortilleria ""PROMADU""",24.007644,-104.598206,Tamaulipas
28,Tortilleria America,27.474354,-99.531627,Tamaulipas
29,Tortilleria R.Maya,29.093807,-110.947896,Tamaulipas
30,Tortilleria TortiMass,22.242845,-97.849236,Tamaulipas
32,Tortillería tommy,22.906206,-98.075213,Tamaulipas
33,Tortillería de Harina “La Pecosita”,24.783773,-107.506774,Tamaulipas
34,"Tortillería ""El Maizal""",23.759553,-99.153458,Tamaulipas
35,TORTILLERIA SAGITARIO,23.715161,-99.091550,Tamaulipas
36,Tortilleria La Palma,23.711877,-99.182114,Tamaulipas
37,"Tortilleria de harina ""Rodríguez""",27.461550,-99.567198,Tamaulipas


In [32]:
# Configure the map plot
map_plot_tort_4 = tortillerias_df_dropped.hvplot.points(
    "lng",
    "lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500 ,
    scale = 5,
    color = "name",
)

# Display the map plot
map_plot_tort_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lng,lat]   (name)

In [33]:
#-------------------------------------------------------------------------------------------------------------------
# In this part we will generate a MAP with FOLIUM library to be able to zoom in and out the map
# on the "tortillerias" near the 7 cities with highest tortilla price
#-------------------------------------------------------------------------------------------------------------------
map_center = [tortillerias_df_dropped['lat'].mean(), tortillerias_df_dropped['lng'].mean()]
map_tortillerias = folium.Map(location=map_center, zoom_start=5.4)

# Adding points to each place
for _, row in tortillerias_df_dropped.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['name'],  
        icon=folium.Icon(color='blue')  
    ).add_to(map_tortillerias)

# Mostrar el mapa
map_tortillerias

In [ ]:
#-------------------------------------------------------------------------------------------------------------------
# In this part we will connect to Google Maps API in order to get data from "Tortillerias" near the cities
# that we are interested in. In this case those will be the 7 cities with highest tortilla price in 2024
#-------------------------------------------------------------------------------------------------------------------
locations = []

# Function to check if a location is within Mexico's bounds
def is_within_mexico(lat, lng):
    mexico_bounds = {
        'lat_min': 14.5320,  # Minimum latitude
        'lat_max': 32.7187,  # Maximum latitude
        'lng_min': -118.4043, # Minimum longitude
        'lng_max': -86.7106   # Maximum longitude
    }
    
    return (mexico_bounds['lat_min'] <= lat <= mexico_bounds['lat_max']) and (mexico_bounds['lng_min'] <= lng <= mexico_bounds['lng_max'])

# Looping through the rows of top_7_cities to fetch tortillerías in a radious of 20000 m
for index, row in bottom_7_cities.iterrows():
    estado = row['State.1']
    coords = (row['Lat'], row['Lon'])
    query = f'tortillería {estado}'
    
    # Query the Google Places API for tortillerías in the area
    results = gmaps.places(query=query, location=coords, radius=20000, region="MX")
    
    for place in results.get('results', []):
        lat = place['geometry']['location']['lat']
        lng = place['geometry']['location']['lng']
        
        # Check if the location is within the bounds of Mexico
        if is_within_mexico(lat, lng):  # Filter by Mexico's limits
            locations.append({
                'name': place['name'],
                'lat': lat,
                'lng': lng,
                'estado': estado
            })
        else:
            print(f"Filtered out place: {place['name']} at ({lat}, {lng}) - Outside of Mexico")

# Create a DataFrame for tortillerías
tortillerias_df_2 = pd.DataFrame(locations)

# Display the resulting DataFrame
tortillerias_df_2

In [ ]:
#-------------------------------------------------------------------------------------------------------------------
# In this part we will generate a MAP with FOLIUM library to be able to zoom in and out the map
# on the "tortillerias" near the 7 cities with lowest tortilla price
#-------------------------------------------------------------------------------------------------------------------
map_center = [tortillerias_df_2['lat'].mean(), tortillerias_df_2['lng'].mean()]
map_tortillerias_2 = folium.Map(location=map_center, zoom_start=6.5)

# Adding points to each place
for _, row in tortillerias_df_2.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['name'], 
        icon=folium.Icon(color='blue')  
    ).add_to(map_tortillerias_2)

map_tortillerias_2